In [1]:
import pandas as pd
from config import masterpassword
from sqlalchemy import create_engine

In [2]:
fifa = pd.read_csv('resources/fifa20_data.csv', encoding='utf-8')
epl = pd.read_csv('resources/players_1920_fin.csv', encoding='utf-8')

c:\users\jplum\anaconda3\envs\pythondata\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
epl.head()

,Unnamed: 0,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,full,team,ppm
0,1,0,0,1,0,0.1,534,37,1,0,...,1,0,0,0,4.5,False,0,Aaron Connolly,Brighton,0.222222
1,2,0,0,1,0,0.3,534,43,1,0,...,1,10589,13500,2911,4.5,True,0,Aaron Connolly,Brighton,0.222222
2,3,0,0,1,0,4.8,534,58,0,0,...,1,8090,11749,3659,4.5,False,0,Aaron Connolly,Brighton,0.222222
3,4,0,0,2,0,0.6,534,63,1,0,...,1,8437,13439,5002,4.5,False,0,Aaron Connolly,Brighton,0.222222
4,5,0,3,53,1,23.8,534,72,0,2,...,13,2156,5952,3796,4.5,True,0,Aaron Connolly,Brighton,2.888889


In [4]:
fifa.head()

,Name,Image,Country,Position,Age,Overall,Potential,Club,ID,Height,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,Lionel Messi,https://cdn.sofifa.org/players/4/20/158023.png,Argentina,"RW,CF,ST",32,94,94,FC Barcelona,158023,"5'7""",...,Medium,Low,5,87,92,92,96,39,66,585
1,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.org/players/4/20/20801.png,Portugal,"ST,LW",34,93,93,Juventus,20801,"6'2""",...,High,Low,5,90,93,82,89,35,78,448
2,Neymar da Silva Santos Jr.,https://cdn.sofifa.org/players/4/20/190871.png,Brazil,"LW,CAM",27,92,92,Paris Saint-Germain,190871,"5'9""",...,High,Medium,5,91,85,87,95,32,58,432
3,Jan Oblak,https://cdn.sofifa.org/players/4/20/200389.png,Slovenia,GK,26,91,91,Atlético Madrid,200389,"6'2""",...,Medium,Medium,3,87,92,78,89,52,90,240
4,Kevin De Bruyne,https://cdn.sofifa.org/players/4/20/192985.png,Belgium,"CAM,CM",28,91,91,Manchester City,192985,"5'11""",...,High,High,4,76,86,92,86,61,78,298


# What to look for:

- EPL
    - Aggregate/group
    - goals and assists
    - red and yellow
    - Club
    
- FIFA
    - Age
    - Position
    - Overall
    - Potential
    - Country
    - Height
    - Value

In [5]:
epl = epl[['full', 'team','goals_scored', 'assists', 'yellow_cards', 'red_cards']]
epl_group = epl.groupby('full').sum()
epl_df = pd.DataFrame(epl_group).reset_index()

epl_df

,full,goals_scored,assists,yellow_cards,red_cards
0,?aglar S?y?nc?,1,1,4,1
1,?rjan Nyland,0,0,0,0
2,Aaron Connolly,3,3,0,0
3,Aaron Cresswell,3,0,7,0
4,Aaron Lennon,0,0,4,0
...,...,...,...,...,...
661,Yerry Mina,2,1,3,0
662,Yoshinori Muto,0,0,0,0
663,Youri Tielemans,3,6,2,0
664,Yves Bissouma,1,0,4,0


In [6]:
# Shrink FIFA dataframe into more manageable data
fifa = fifa[['Name', 'Position', 'Country', 'Overall', 'Potential', 'Height', 'Value']]
fifa.head()

,Name,Position,Country,Overall,Potential,Height,Value
0,Lionel Messi,"RW,CF,ST",Argentina,94,94,"5'7""",€95.5M
1,C. Ronaldo dos Santos Aveiro,"ST,LW",Portugal,93,93,"6'2""",€58.5M
2,Neymar da Silva Santos Jr.,"LW,CAM",Brazil,92,92,"5'9""",€105.5M
3,Jan Oblak,GK,Slovenia,91,91,"6'2""",€77.5M
4,Kevin De Bruyne,"CAM,CM",Belgium,91,91,"5'11""",€90M


In [7]:
# Use regex to replace bad character values in EPL dataframe
import re

# Find broken names
broken_list = []
for i in epl_df['full']:
    if '?' in i:
        broken_list.append(i)
broken_list

# Regex search and replace
for name in epl_df['full']:
    for player in fifa['Name']:
        pattern = name.replace('?', '\S')
        if re.search(pattern, player):
            epl_df['full'].loc[epl_df['full'] == name] = player

c:\users\jplum\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
# connection_string = f"postgres:{masterpassword}@localhost:5432/postgres"
# engine = create_engine("postgres://postgres@/postgres")

connection_string = f"postgres:{masterpassword}@localhost:5432/postgres"
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
connection = engine.connect()

connection.execute("commit")

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
connection.execute("create database etl_project")

connection.close()

In [ ]:
connection_string = f"postgres:{masterpassword}@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
fifa.set_index("Name", inplace=True)

In [ ]:
fifa.to_sql(name='fifa', con=engine, if_exists='append', index=True)

In [ ]:
epl_df.to_sql(name='epl', con=engine, if_exists='append', index=True)